In [76]:
import math
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [77]:
# read in Vicky's merged data
data_file = "/Users/allieliu/Compgen/merged.csv"
raw_data = pd.read_csv(data_file)

In [78]:
# 322 gene list is not available, maybe from "Consensus genes of the literature to predict breast cancer recurrence"?
# so, I downloaded the 41 gene list from CBCG website https://cbcg.dk/causal.html just to see results
# Following the paper, I fed 41 gene list to Genecodis, which grouped these 41 genes into 191 gene sets (more than 1 gene in the set)

cbcg41_pd = pd.read_csv('/Users/allieliu/Compgen/enrich-input1-GO_BP.tsv',sep = '\t')
cbcg41_list_raw = cbcg41_pd['genes'].tolist()
cbcg41_sets = []

for s in cbcg41_list_raw:
    s = s.split(', ')
    #make sure there are more than one gene in any set to avoid zero standard deviation
    if len(s) > 1:
        cbcg41_sets.append(s)
print(len(cbcg41_sets))

gene_sets = cbcg41_sets
gene_list = list(set([ i for sub_list in gene_sets for i in sub_list]))
other_cols = ['PATIENT_ID','high_risk']
data = raw_data[gene_list + other_cols].copy()
data.head()

191


,MRE11,BARD1,CDH1,RECQL,BRIP1,ERBB2,RAD50,TP53,PPM1D,CHEK2,ABRAXAS1,RAD51D,STK11,NF1,RAD51C,MEN1,BLM,PMS2,FANCD2,BRCA1,ATRIP,MSH2,POLG,MSH6,TEX15,ATM,FANCM,XRCC2,NBN,RINT1,FANCC,RBBP8,MCPH1,PALB2,RAD51B,BRCA2,APC,HOXB13,PTEN,MUTYH,PATIENT_ID,high_risk
0,-0.246463,0.071265,-0.573273,-0.412900,-0.011822,-0.235215,0.158942,-0.256023,0.228091,-0.462922,0.463391,0.013450,0.681559,-0.577340,-0.072343,-0.563199,-0.509627,-0.385463,-0.128549,-0.478987,-0.251623,-0.269315,-0.624045,-0.516354,-0.455436,-0.023861,-0.234855,-0.137137,-0.321343,0.392688,-0.263327,-0.445321,0.681927,-0.551647,-0.203753,0.180453,-0.413404,0.197442,0.171042,0.003313,MB-0020,True
1,0.337672,0.332016,-0.614060,-0.354264,0.380572,0.244103,0.556372,0.264406,-0.723315,-0.083016,-0.141804,-0.604564,0.220653,0.276187,-0.688045,-0.176411,-0.524062,0.614513,0.307379,-0.487063,0.113103,0.629528,0.181523,0.497250,0.093756,-0.548182,-0.008956,0.301593,0.309782,0.632868,-0.192252,-0.645777,-0.099725,0.073542,-0.287742,-0.364155,-0.374120,-0.329758,0.534497,-0.607693,MB-0035,True
2,0.539950,-0.225079,0.085593,0.497798,0.430311,-0.434911,-0.052125,0.160366,0.214626,-0.016667,-0.196044,0.012507,-0.201762,-0.135916,-0.014134,-0.659463,0.264156,-0.099699,-0.222737,0.474909,-0.344867,-0.356104,0.142298,-0.492875,0.243624,0.726388,0.209425,-0.242088,-0.061739,-0.003479,-0.530461,-0.283773,0.479500,-0.061543,-0.277838,-0.547658,0.131455,0.091747,0.376713,-0.194496,MB-0045,False
3,-0.657282,0.128691,0.175368,0.700450,0.865004,-0.703281,-0.745088,-0.659537,0.657107,0.846021,0.701640,0.234842,0.578902,-0.736772,0.654738,-0.837898,0.390989,0.902703,0.172141,-0.750935,-0.696660,0.932405,-0.725692,0.683121,0.552877,0.220842,0.723994,-0.069065,0.745410,0.699679,0.516302,0.639126,-0.430795,0.448515,0.074027,0.829390,-0.229700,0.465733,-0.314080,-0.458815,MB-0062,False
4,-0.317853,0.214952,-0.076447,0.648233,0.743478,-0.548903,-0.531895,0.506049,0.557806,0.820129,0.727943,-0.444107,-0.433216,-0.698343,0.368145,-0.108509,0.546633,0.844498,0.079325,0.080640,-0.589918,0.914898,-0.366559,0.715847,-0.405271,0.519154,0.479131,0.201089,0.521991,0.158601,-0.061686,-0.191340,-0.433659,-0.742950,-0.062549,0.824897,0.628938,0.196581,-0.102359,-0.704042,MB-0079,True


In [79]:
# calculate t-statistic for each gene set
module_list = []

for i in range(len(gene_sets)):
    module_name = 'Module_' + str(i + 1)
    module_list.append(module_name)
    n = len(gene_sets[i])
    tmp = data[gene_sets[i]].copy()
    
    #make sure there are more than one gene in any set to avoid zero standard deviation
    data[module_name] = math.sqrt(n) * tmp.mean(axis = 1, skipna = True) / tmp.std(axis = 1, skipna = True)

In [80]:
#rank modules
rank_data = pd.DataFrame([])

for i in range(data.shape[0]):
    tmp = data[module_list + other_cols].iloc[i:i + 1,:].copy()
    tmp['key'] = 1
    tmp_t = tmp[module_list].transpose()
    tmp_t['rank'] = tmp_t[i].rank(method = 'first')#avoid tied values
    tmp_back = tmp_t.transpose().iloc[1:2,:]
    tmp_back['key'] = 1
    tmp_back = tmp_back.merge(tmp[other_cols + ['key']],how = 'inner',on = 'key').drop(columns = ['key'])
    rank_data = pd.concat([rank_data, tmp_back], ignore_index = True)
    
rank_list = rank_data.values.tolist()

In [81]:
# K-fold cross-validation


# splitting rank data into folds
k_fold = 10

n = rank_data.shape[0]
n_valid = n / k_fold
n_train = n - n_valid
rank_data['random'] = np.random.randint(n)
rank_data = rank_data.sort_values('random')
rank_data = rank_data.reset_index().drop(columns = ['index'])
rank_data['training'] = 1
rank_data.loc[rank_data.index >= n_train, 'training'] = 0

print(rank_data['training'].value_counts())

train_data_h = rank_data[(rank_data['training'] == 1) & (rank_data['high_risk'] == True)].drop(columns = ['random','training'])
train_data_l = rank_data[(rank_data['training'] == 1) & (rank_data['high_risk'] == False)].drop(columns = ['random','training'])
validation_data = rank_data[rank_data['training'] == 0].drop(columns = ['random','training'])

1    688
0     76
Name: training, dtype: int64


In [82]:
# training the data
def hmm_rank(train_data, n_state):
    n = train_data.shape[0]
    n_emission = len(gene_sets)
    
    emiss_probs = np.zeros((n_state, n_emission))
    
    for i in range(n_state):
        for j in range(n_emission):
            emiss_probs[i][j] = sum(train_data['Module_' + str(j + 1)] == (i + 1)) / n

    return emiss_probs
    
eh = hmm_rank(train_data_h, 6) # train one on high, one on low risk data
el = hmm_rank(train_data_l, 6)

In [83]:
# prediction
def predict(new_list, emission_h, emission_l):
    n_state = len(emission_h)
    n = n_state
    
    start_probs = np.zeros(n_state)
    start_probs[0] = 1
    trans_probs = [[ 1 if j == i + 1 else 0 for j in range(n_state)] for i in range(n_state)]

    fh = np.zeros((n_state, n))
    fl = np.zeros((n_state, n))
    
    for t in range(n):
        for k in range(n_state):
            if t == 0:
                fh[k][t] = start_probs[k]
                fl[k][t] = start_probs[k]
            else:
                fh[k][t] = sum(fh[i][t-1] * trans_probs[i][k] for i in range(n_state))
                fl[k][t] = sum(fl[i][t-1] * trans_probs[i][k] for i in range(n_state))
            
            j=new_list.index(k+1)
            fh[k][t] *= emission_h[k][j]
            fl[k][t] *= emission_l[k][j]
            
    loglik_h = np.log(sum(fh[k][n-1] for k in range(n_state)))
    loglik_l = np.log(sum(fl[k][n-1] for k in range(n_state)))
    
    # chooses high or low risk based on which one has a higher log lik
    if loglik_h > loglik_l:
        return True, loglik_h 
    else:
        return False, loglik_l

validation_list = validation_data.values.tolist()


for i in range(len(validation_list)):
    new_list = validation_list[i]
    high_risk, p = predict(new_list, eh, el)
    validation_list[i].append(high_risk)
    validation_list[i].append(p)
pred = pd.DataFrame(validation_list, columns = validation_data.columns.tolist()+['prediction', 'loglik'])


#confusion matrix to see performance
pd.crosstab(pred['prediction'],pred['high_risk'])

high_risk,False,True
prediction,,
False,43,18
True,6,9


In [87]:
# MCC & AUC model evaluation    
TP = sum((pred['high_risk'] == True) & (pred['prediction'] == True))
TN = sum((pred['high_risk'] == False) & (pred['prediction'] == False))
FN = sum((pred['high_risk'] == True) & (pred['prediction'] == False))
FP = sum((pred['high_risk'] == False) & (pred['prediction'] == True))
MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

n_auc_curve = 10
pred = pred.sort_values('loglik')
size = int(pred.shape[0]/n_auc_curve)
start_index = 0
tpr = []
fpr = []

for i in range(n_auc_curve):
    end_index = min((i + 1) * size,pred.shape[0] - 1)
    partition = pred.iloc[start_index:end_index+1]
    TP = sum((partition['high_risk'] == True) & (partition['prediction'] == True))
    TN = sum((partition['high_risk'] == False) & (partition['prediction'] == False))
    FN = sum((partition['high_risk'] == True) & (partition['prediction'] == False))
    FP = sum((partition['high_risk'] == False) & (partition['prediction'] == True))
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0  # Sensitivity
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  # 1 - Specificity
    tpr.append(TPR)
    fpr.append(FPR)
    start_index = end_index + 1
    
# Sort by FPR for correct integration order
fpr, tpr = zip( * sorted(zip(fpr, tpr)))

# calculate AUC using trapezoidal rule
AUC = np.trapz(tpr, fpr)
print(AUC)
print(MCC)

0.17500000000000002
0.2535792623389988
